In [ ]:
# !pip install accelerate -U
# !pip install transformers[torch]
# !pip install --upgrade evaluate jiwer
# !pip install transformers==4.38.2
# !pip install datasets==2.18.0
# !pip install soundfile
# !pip install librosa

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
common_dataset = DatasetDict()

common_dataset['train'] = load_dataset('mozilla-foundation/common_voice_13_0', 'ta', split='train[:10%]')
common_dataset['test'] = load_dataset('mozilla-foundation/common_voice_13_0', 'ta', split='test[:10%]')

common_dataset

/usr/local/lib/python3.9/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset fr

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4335
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1197
    })
})

In [ ]:
common_dataset = common_dataset.select_columns(['audio', 'sentence'])
common_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4335
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1197
    })
})

In [ ]:
import IPython as ipy

In [ ]:
sample = common_dataset['train'][0]['audio']
ipy.display.Audio(sample['array'], rate=sample['sampling_rate'])

In [ ]:
from transformers import WhisperProcessor

In [ ]:
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language='tamil',
    task='transcribe'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
common_dataset['train'].features

{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [ ]:
from datasets import Audio

In [ ]:
sampling_rate = processor.feature_extractor.sampling_rate
common_dataset = common_dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
common_dataset['train'].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [ ]:
def prepare_dataset(example):
  audio = example['audio']
  example = processor(
      audio=audio['array'],
      sampling_rate=audio['sampling_rate'],
      text=example['sentence']
      )
  example['input_length'] = len(audio['array'])/audio['sampling_rate']
  return  example


In [ ]:
common_dataset = common_dataset.map(prepare_dataset,
                                    remove_columns=common_dataset.column_names['train'], num_proc=2)
common_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 4335
    })
    test: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 1197
    })
})

In [ ]:
max_input_length = 30.0

def is_audio_in_length_range(length):
  return length < max_input_length

In [ ]:
common_dataset['train'] = common_dataset['train'].filter(
    is_audio_in_length_range,
    input_columns=['input_length']
)

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate
metric = evaluate.load('wer')

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

def compute_metrics(preds):
  pred_ids = preds.predictions
  label_ids = preds.label_ids

  label_ids[label_ids==-100] = processor.tokenizer.pad_token_id

  pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
  label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

  wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

  pred_str_norm = [normalizer(pred) for pred in  pred_str]
  label_str_norm = [normalizer(pred) for pred in  label_str]

  pred_str_norm = [ele for ele in  pred_str_norm if len(ele) > 0]
  label_str_norm = [ele for ele in  label_str_norm if len(ele) > 0]

  wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

  return {
      'wer_ortho': wer_ortho,
      'wer': wer
  }

In [ ]:
from transformers import WhisperForConditionalGeneration

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(
    'openai/whisper-small'
)

In [ ]:
from functools import partial

In [ ]:
model.config.use_cache = False

In [ ]:
model.generate = partial(model.generate, language='tamil', task='transcribe', use_cache=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./whisper-small-tamil',
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=10,
    max_steps=100,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    dataloader_num_workers=5,
    dataloader_prefetch_factor=5,
    report_to=None
)


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_dataset['train'],
    eval_dataset=common_dataset['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor
)

/usr/local/lib/python3.9/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

wandb: Currently logged in as: carlfeynman. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_13_0",
    "dataset": "Common Voice 13", 
    "language": "tamil",
    "model_name": f"carl-whisper-small-finetuned-tamil",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
trainer.push_to_hub(**kwargs)